In [27]:
# This code block stores file paths to variables to make the code neat
# The stored variables are called in the read_excel function and stored as dataframes

import pandas as pd
import numpy as np

# Store excel file locations to variables (change it as per your path to file)
file_2012=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\11. Section 2 Part 6 Income from Livestock\2012_s2p6_f.csv"
file_2013=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\11. Section 2 Part 6 Income from Livestock\2013_s2p6_f.csv"
file_2014=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\11. Section 2 Part 6 Income from Livestock\2014_s2p6_f.csv"

# Read excel files
df_2012 = pd.read_csv(file_2012)
df_2013 = pd.read_csv(file_2013)
df_2014 = pd.read_csv(file_2014)


In [28]:
'''
This code block will be used to standardize column names across the years to avoid discrepancies during the merging process.
For example in the roaster data for 2013 rq21 and rq23 are not the same as rq21 and rq23 in 2014 data, but they have the same variable names. Hence, we decide to rename such columns beforehand
We will add the updated name to the mapping dictionaries instead of the original names.

'''

# Rename columns in df_2012
df_2012.rename(columns={
    'item': 's2p6_sr',
    'S2P6Q1': 's2p6_q1',
    'S2P6Q3A_QTY': 's2p6_q3a_qty',
    'S2P6Q3A_VAL': 's2p6_q4',
    'S2P6Q3B': 's2p6_q5',
    'S2P6Q4': 's2p6_q6',
    'S2P6Q8_QTY': 's2p6_q7',
    'S2P6Q9': 's2p6_q9',
    'S2P6Q5': 's2p6_q10',
    'S2P6Q6': 's2p6_q11',
    'S2P6Q7': 's2p6_q12',
    'PROVINCE_ID': 'P_ID',
    'DISTRICT_ID': 'D_ID',
    'TEHSIL_ID': 'T_ID',
    'UC_ID': 'UC_ID',
    'MAUZA_ID': 'M_ID'
}, inplace=True)

# Rename columns in df_2013
df_2013.rename(columns={
    's2p6_q3a_unit': 's2p6_q3b_unit',
    's2p6_q3c': 's2p6_q4',
    's2p6_q3b': 's2p6_q5',
    's2p6_q4': 's2p6_q6',
    's2p6_q8_qty': 's2p6_q7',
    's2p6_q8_unit': 's2p6_q8',
    's2p6_q9': 's2p6_q9',
    's2p6_q5': 's2p6_q10',
    's2p6_q6': 's2p6_q11',
    's2p6_q7': 's2p6_q12'
}, inplace=True)

# df_2014 doesn't need renaming as it is the reference

In [29]:
# Updated mappings

mapping_2012 = [
    'hid', 'round', 's2p6_sr', None, 's2p6_q1', None, 's2p6_q3a_qty', None, 's2p6_q4', 's2p6_q5',
    's2p6_q6', 's2p6_q7', None, 's2p6_q9', 's2p6_q10', 's2p6_q11', 's2p6_q12',
    'P_ID', 'D_ID', 'T_ID', 'UC_ID', 'M_ID'

]

mapping_2013 = [
    'hid', 'round', 's2p6_sr', 's2p6_items', 's2p6_q1', 's2p6_q2', 's2p6_q3a_qty',
    's2p6_q3b_unit', 's2p6_q4', 's2p6_q5', 's2p6_q6', 's2p6_q7', 's2p6_q8',
    's2p6_q9', 's2p6_q10', 's2p6_q11', 's2p6_q12', None, None,
    None, None, None
]

mapping_2014 = [
    'hid', 'round', 's2p6_sr', 's2p6_items', 's2p6_q1', 's2p6_q2', 's2p6_q3a_qty',
    's2p6_q3b_unit', 's2p6_q4', 's2p6_q5', 's2p6_q6', 's2p6_q7', 's2p6_q8',
    's2p6_q9', 's2p6_q10', 's2p6_q11', 's2p6_q12', None, None,
    None, None, None
]


In [30]:
# Create a list of all possible columns in the correct order
all_columns = []

for col in mapping_2012:
    if col and col not in all_columns:
        all_columns.append(col)   
for col in mapping_2013:
    if col and col not in all_columns:
        all_columns.append(col)
for col in mapping_2014:
    if col and col not in all_columns:
        all_columns.append(col)



In [31]:
def standardize_and_merge(dfs, mappings, all_columns):
    merged_data = {col: [] for col in all_columns}

    for df, mapping in zip(dfs, mappings):
        print(f"Processing DataFrame with columns: {df.columns.tolist()}")
        for i, col in enumerate(mapping):
            if col:
                ref_col = col.strip()  # Remove leading/trailing whitespace
                if ref_col not in merged_data:
                    merged_data[ref_col] = []
                if ref_col in df.columns:
                    print(f"Appending data for column {ref_col}")
                    if isinstance(df[ref_col], pd.Series):
                        merged_data[ref_col].extend(df[ref_col].tolist())
                    elif isinstance(df[ref_col], pd.DataFrame):
                        print(f"Column {ref_col} is duplicated in DataFrame. Appending data for each duplicate.")
                        for _, series in df[ref_col].items():
                            merged_data[ref_col].extend(series.tolist())
                else:
                    print(f"Column {ref_col} not found in DataFrame. Adding NaNs.")
                    merged_data[ref_col].extend([np.nan] * len(df))
    
    max_len = max(len(v) for v in merged_data.values())
    for key in merged_data:
        col_len = len(merged_data[key])
        if col_len < max_len:
            merged_data[key].extend([np.nan] * (max_len - col_len))

    merged_df = pd.DataFrame.from_dict(merged_data)
    return merged_df


In [32]:
# Usage with dataframes and mappings
dfs = [df_2012, df_2013, df_2014]
mappings = [mapping_2012, mapping_2013, mapping_2014]

merged_df = standardize_and_merge(dfs, mappings, all_columns)

Processing DataFrame with columns: ['Unnamed: 0.1', 'Unnamed: 0', 'hid', 'round', 's2p6_sr', 's2p6_q1', 's2p6_q3a_qty', 's2p6_q4', 's2p6_q5', 's2p6_q6', 's2p6_q10', 's2p6_q11', 's2p6_q12', 's2p6_q7', 's2p6_q9', 'P_ID', 'D_ID', 'T_ID', 'UC_ID', 'M_ID']
Appending data for column hid
Appending data for column round
Appending data for column s2p6_sr
Appending data for column s2p6_q1
Appending data for column s2p6_q3a_qty
Appending data for column s2p6_q4
Appending data for column s2p6_q5
Appending data for column s2p6_q6
Appending data for column s2p6_q7
Appending data for column s2p6_q9
Appending data for column s2p6_q10
Appending data for column s2p6_q11
Appending data for column s2p6_q12
Appending data for column P_ID
Appending data for column D_ID
Appending data for column T_ID
Appending data for column UC_ID
Appending data for column M_ID
Processing DataFrame with columns: ['Unnamed: 0', 'hid', 'round', 's2p6_sr', 's2p6_items', 's2p6_q1', 's2p6_q2', 's2p6_q3a_qty', 's2p6_q3b_unit', 's

In [33]:
# Rename columns for the merged file (if needed)
rename_mapping = {
   'hid': 'HID',
    'round': 'Survey_Round',
    's2p6_sr': 'LF_Item_code',
    's2p6_items': 'LF_Item_name',
    's2p6_q1': 'LF_Item_HH_P',
    's2p6_q2': 'LF_Time_Prd',
    's2p6_q3a_qty': 'LF_Qty_P',
    's2p6_q3b_unit': 'LF_Unit_P',
    's2p6_q4': 'LF_Val_T_P',
    's2p6_q5': 'LF_Item_P_Mon',
    's2p6_q6': 'LF_Item_S',
    's2p6_q7': 'Item_S_Qty',
    's2p6_q8': 'Item_S_Qty_Unit',
    's2p6_q9': 'Item_S_PPU',
    's2p6_q10': 'Item_S_pers',
    's2p6_q11': 'Item_S_Loc',
    's2p6_q12': 'Item_S_Dist'
}

merged_df.rename(columns=rename_mapping, inplace=True)
'''
# Drop redundant columns
merged_df.drop(merged_df.columns[merged_df.columns.str.contains('Unnamed', case=True)], axis=1, inplace=True)
merged_df.drop(merged_df.columns[merged_df.columns.str.contains(' ', case=False)], axis=1, inplace=True)
'''
# Save the merged dataframe to a CSV file
merged_df.to_csv('9. merged_merged_EnI_NAE Income From Livestock Farming.csv', index=False)
